In [ ]:
## このプログラムは「House Prices: Advanced Regression Techniques」
## において作成したノートブックにおいて動作します

# 訓練データとテストデータをデータフレームに読み込む
import pandas as pd
train = pd.read_csv(
    "../input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv(
    "../input/house-prices-advanced-regression-techniques/test.csv")
print('train shape:', train.shape) # 訓練データの形状を出力
print('test shape:', test.shape)   # テストデータの形状を出力

In [ ]:
# 訓練データの情報を出力
train.info()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
%matplotlib inline

# SalePriceについて底をeとするa+1の対数に変換し、
# 元の値と共にデータフレームに登録MiscVal
prices = pd.DataFrame({'price':train['SalePrice'],
                       'log(price + 1)':np.log1p(train['SalePrice'])})
print(prices, '\n')
# 'price'の対数変換前後の歪度を出力
print('price skew       :', skew(prices['price']))
print('log(price+1) skew:', skew(prices['log(price + 1)']))

# "SalePrice"の変換前と変換後をヒストグラムにする
# プロット図のサイズを設定
plt.rcParams['figure.figsize'] = (12.0, 6.0)
prices.hist()

In [ ]:
# SalePriceの値を、底をeとするa+1の対数に変換する
train["SalePrice"] = np.log1p(train["SalePrice"])

In [ ]:
# 訓練データとテストデータからMSSubClass～SaleConditionのカラムのみを抽出して連結
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))
# 連結したデータを出力
all_data

In [ ]:
# 数値型のカラムで歪度が0.75を超える場合は対数変換する

from scipy.stats import skew

# object型ではないカラムのインデックスを取得
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# object型ではないカラムの歪度を、欠損値を除いてから求める
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna()))
print('-----Skew Non-object-type-----')
print(skewed_feats)

# 歪度が0.75より大きいカラムのみをskewed_featsに再代入
skewed_feats = skewed_feats[skewed_feats > 0.75]
print('-----Skewness greater than 0.75-----')
print(skewed_feats)
# 抽出したカラムのインデックスを取得
skewed_feats = skewed_feats.index

# 歪度が0.75より大きいカラムの値を対数変換する
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])
all_data[skewed_feats] # 歪度が0.75より大きいカラムの対数変換後を出力

In [ ]:
# LotShape(土地の形状)を情報表現（ダミー変数）に変換
cc_data = pd.get_dummies(train['LotShape'])
# 元の'LotShape'を追加
cc_data['LotShape'] = train['LotShape']
# 20レコードを出力
cc_data[:20]

In [ ]:
# カテゴリカルなカラムを情報表現（ダミー変数）に変換
all_data = pd.get_dummies(all_data)

In [ ]:
# 欠損値NaNをそのカラムの平均値に置き換える(平均は訓練データから求める)
all_data = all_data.fillna(all_data[:train.shape[0]].mean())

In [ ]:
# 訓練データとテストデータに分ける
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train.SalePrice

In [ ]:
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    """二乗平均平方根誤差
    
    Parameters:
      model(obj): Modelオブジェクト
    Returns:
      (float)訓練データの出力値と正解値とのRMSE
    """
    # クロスバリデーションによる二乗平均平方根誤差の取得
    rmse = np.sqrt(
        -cross_val_score(
            model, X_train, y,
            scoring="neg_mean_squared_error", # 平均二乗誤差を指定
            cv = 5))                          # データを5分割
    return(rmse)

In [ ]:
from sklearn.linear_model import Ridge

# リッジ回帰モデルを生成
model_ridge = Ridge()

# L2正則化の強度を10パターン用意
alphas = [0.05, 0.1, 0.5, 1, 5, 10, 15, 30, 50, 75]
# 正則化の各強度でリッジ回帰を実行
# 5分割のクロスバリデーションでRMSEを求め、その平均を取得
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]

# cv_ridgeをSeriesオブジェクトに変換
cv_ridge = pd.Series(cv_ridge, index = alphas)
# スコアを出力
print('Ridge RMSE loss:')
print(cv_ridge, '\n')
# スコアの平均を出力
print('Ridge RMSE loss Mean:')
print(cv_ridge.mean())

# 正則化の強度別のスコアをグラフにする
plt.ﬁgure(ﬁgsize=(10, 5)) # 描画エリアのサイズ
plt.plot(cv_ridge)        # cv_ridgeをプロット
plt.grid()                # グリッド表示
plt.title('Validation - by regularization strength')
plt.xlabel('Alpha')
plt.ylabel('RMSE')
plt.show()

In [ ]:
from sklearn.linear_model import LassoCV

# ラッソ回帰モデルで推定する
# L1正則化項を4パターンで試す
model_lasso = LassoCV(
    alphas = [1, 0.1, 0.001, 0.0005]).fit(X_train, y)

print('Lasso regression RMSE loss:')                # クロスバリデーションによる
print(rmse_cv(model_lasso))                         # RMSEを出力

print('Average loss:', rmse_cv(model_lasso).mean()) # RMSEの平均を出力
print('Minimum loss:', rmse_cv(model_lasso).min())  # RMSEの最小値を出力
print('Best alpha  :', model_lasso.alpha_)          # 採用されたalpha値を出力

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label = y)
#dtest = xgb.DMatrix(X_test)

# 決定木の深さ3、学習率0.1
params = {"max_depth":3, "eta":0.1}
# xgboostモデルでクロスバリデーションを実行
cross_val = xgb.cv(
    params,
    dtrain,
    num_boost_round=1000,     # 決定木の本数
    early_stopping_rounds=50) # アーリーストッピングの監視回数
cross_val

In [ ]:
# 30回以降の検証データと訓練データのRMSEをグラフにする
plt.ﬁgure(ﬁgsize=(8, 6)) # 描画エリアのサイズ
plt.plot(cross_val.loc[30:,["test-rmse-mean", "train-rmse-mean"]])
plt.grid()               # グリッド表示
plt.xlabel('num_boost_round')
plt.ylabel('RMSE')
plt.show()

In [ ]:
# xgboostで学習する
model_xgb = xgb.XGBRegressor(
    n_estimators=410,  # 決定木の本数
    max_depth=3,       # 決定木の深さ
    learning_rate=0.1) # 学習率0.1
model_xgb.fit(X_train, y)

print('xgboost RMSE loss:')
print(rmse_cv(model_xgb).mean())

In [ ]:
lasso_preds = np.expm1(model_lasso.predict(X_test))
xgb_preds = np.expm1(model_xgb.predict(X_test))

In [ ]:
preds = lasso_preds*0.7 + xgb_preds*0.3

In [ ]:
solution = pd.DataFrame({"id":test.Id, "SalePrice":preds})
solution.to_csv("ridge_sol.csv", index = False)